In [1]:
# Librerias
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
### CONSUMIR EXCEL DE LIMPIEZA ###

In [3]:
#funcion para leer data

def leerExcel(año):
    dataAño = pd.read_excel("Data-resoluciones.xlsx",
                             sheet_name=año,
                             usecols=[0,1,2],
                             header=None)
    return dataAño

In [4]:
#Concatenando valores de Excel de cada año
DataTotal = pd.concat([leerExcel("2006"), leerExcel("2007"), 
                       leerExcel("2008"), leerExcel("2009"),
                       leerExcel("2010"), leerExcel("2011"),
                      leerExcel("2012"),leerExcel("2013"),
                      leerExcel("2014"),leerExcel("2015")],ignore_index=True)

In [13]:
##VERSION 2
## Creando un diccionario que convierta a lista las normativas (para luego extraerlas)

resoluciones = {}
num_res = 0

filas = len(DataTotal)
variables = [ 'Número de resolución',
             'Empresa denunciada',
             'Normativa sectorial aplicada','Otras normas','Normativa específica',
             'Configuración del Tribunal',
             'Decisión']

for fila in range(filas):
    # Cada resolución será un diccionario dentro del diccionario

    # Debe crearse una resolución si es la primera fila o si la última fila revisada fue "Decisión"
    if fila == 0 or DataTotal.iloc[fila-1,0] == 'Decisión':
        num_res += 1
        resoluciones[num_res] = {}

    # Si en la primera fila hay una variable, creamos una llave en el diccionario de la resolución
    if DataTotal.iloc[fila, 0] in variables:
        variable = DataTotal.iloc[fila, 0]
        if variable in variables:
            resoluciones[num_res][variable] = DataTotal.iloc[fila, 1]

    if pd.isna(DataTotal.iloc[fila, 0]) and pd.notna( DataTotal.iloc[fila, 1] ):
        try:
            resoluciones[num_res][variable] += " " + str(DataTotal.iloc[fila, 1])
        except:
            pass



In [ ]:
##VERSION 2
## Creando un diccionario que convierta a lista las normativas (para luego extraerlas)

resoluciones = {}
num_res = 0
valores=[]

filas = len(DataTotal)
variables = [ 'Número de resolución',
             'Empresa denunciada',
             'Normativa sectorial aplicada','Otras normas','Normativa específica',
             'Configuración del Tribunal',
             'Decisión']

for fila in range(filas):
    # Cada resolución será un diccionario dentro del diccionario

    # Debe crearse una resolución si es la primera fila o si la última fila revisada fue "Decisión"
    if fila == 0 or DataTotal.iloc[fila-1,0] == 'Decisión':
        num_res += 1
        resoluciones[num_res] = {}

    # Si en la primera fila hay una variable, creamos una llave en el diccionario de la resolución
    if DataTotal.iloc[fila, 0] in variables:
        variable = DataTotal.iloc[fila, 0]
        if variable in variables:
            resoluciones[num_res][variable] = DataTotal.iloc[fila, 1]

    if pd.isna(DataTotal.iloc[fila, 0]) and pd.notna( DataTotal.iloc[fila, 1] ):
        try:
            resoluciones[num_res][variable] += " " + str(DataTotal.iloc[fila, 1])
        except:
            pass




In [21]:
#Trayendo solo las normas para contarlas

normativas = ['Normativa sectorial aplicada','Otras normas','Normativa específica']
normas = []
for resolucion in resoluciones:
    for norma in resoluciones[resolucion]:
        if norma in normativas:
            lista = resoluciones[resolucion][norma]
            normas.append(lista)

normas = pd.Series(normas)


In [23]:
type(normas)

pandas.core.series.Series

In [24]:
#contando ocurrencias

countNormas = normas.value_counts()

In [25]:
#convirtiendo a DF las normas y sus ocurrencias

DFNormas = pd.DataFrame(countNormas)
DFNormas

## 84 normas ##

,0
"DECRETO LEGISLATIVO N° 807 Facultades, normas y organización del INDECOPI",83
Código de Protección y Defensa del Consumidor,46
Ley No. 27444 Ley del Procedimiento Administrativo General,39
Código Procesal Civil,11
"DECRETO LEGISLATIVO N° 807 Facultades, normas y organización del INDECOPI Res. 085-96-TDC",11
...,...
"DECRETO LEGISLATIVO N° 807 Facultades, normas y organización del INDECOPI D. Supremo No. 009-2004-MTC Reglamento Nacional de Administración de Transporte D. Supremo 017-2009-MTC",1
"DECRETO LEGISLATIVO N° 807 Facultades, normas y organización del INDECOPI D. Leg 1045 Ley Complementaria al Sistema de Protección al Consumidor Ley No. 27444 Ley del Procedimiento Administrativo General",1
"DECRETO LEGISLATIVO N° 807 Facultades, normas y organización del INDECOPI Ley No. 27809 Ley General del Sistema Concursal Ley No. 27444 Ley del Procedimiento Administrativo General",1
Res. 085-96-TDC Res. 0158-2003/TDC-INDECOPI Ley de Aeronaútica Civil del Perú Pago de costas y costos,1


In [ ]:
#Exportando a csv para validarlo

DFNormas.to_csv("mp-nomr.csv")

In [ ]:
## Extrayendo las normativas con el ID de resoluciones y decisiones (esto luego nos permitira hacer Arbol de dec)

normativas = ['Número de resolución', 'Normativa específica', 'Normativa sectorial aplicada', 'Otras normas',
             "Decisión"]

dictNormativas = {}
for resolucion in resoluciones:
    dictNormativas[resolucion] = {}
    for variable in resoluciones[resolucion]:
        if variable in normativas:
            dictNormativas[resolucion][variable] = resoluciones[resolucion][variable]
            
pprint(dictNormativas)

In [ ]:
normas_decision = pd.Series(dictNormativas)

In [ ]:
pprint(normas_decision)

In [ ]:
## Visualizacion

In [ ]:
import plotly.graph_objects as go

In [ ]:
#filtrando el top 10
DFPlot = DFNormas[:11][:].sort_values(by=[0],ascending=True)
DFPlot

In [ ]:
#realizando ploteo de normas y ocurrencias, no se ve tan bien porq las normas tienen nombres largos 
# y son muchas (58)


my_layout = ({"title": "Normativas más citadas",
                       "yaxis": {"title":"Normativas"},
                       "xaxis": {"title":"Cantidad de citas"},
                       "showlegend": False})



fig = go.Figure(go.Bar(
            x=DFPlot[0],
            y=DFPlot.index,
            orientation='h'),layout = my_layout)


fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
#Arbol

In [ ]:
### ARQUITECTURA: Arbol de decision ####

#plot examples: https://mljar.com/blog/visualize-decision-tree/

def plotArbolito(X,Y):

    def Arbolito(X,Y):
        dt = tree.DecisionTreeClassifier(criterion = 'entropy',random_state=46)
        
        return dt.fit(X, Y)
    
    arbol = Arbolito(X,Y)
    r = export_text(arbol, feature_names=list(X.columns[:]))
    print(r)

In [ ]:
##### DATOS ######

#Entradas
Toda_normativa = pd.read_csv("Toda-normativa.csv",index_col=0,header=1)


#Salidas
dataResol = pd.read_csv("Total-data-resol-1y2Quinquenio.csv")
Y_merge = dataResol[["Número de resolución","Decisión"]]
Y_merge = Y_merge.replace({"Confirmar": 1, "Revocar": 0, "Otros": 2})

Y = dataResol[["Decisión"]]

#  "Confirmar": 1, "Revocar": 0, "Otros": 2
Y = Y.replace({"Confirmar": 1, "Revocar": 0, "Otros": 2})